# Using `numba.jit` to speedup the computation of the Cityblock distance matrix 


In this notebook we implement a function to compute the Cityblock distance matrix using Numba's parallelism and the *just-in-time* compilation decorator.

In [ ]:
import numpy as np
import numba

In [ ]:
@numba.njit()
def cityblock_numba(x, y):
    """Implementation with numba."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r

    return dist_matrix


@numba.njit(parallel=True)
def cityblock_numba_parallel(x, y):
    """Implementation with numba."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in numba.prange(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r

    return dist_matrix

In [ ]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(cityblock_numba_parallel(a, a) - cityblock_numba(a, a)).max())

In [ ]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit cityblock_numba_parallel(x,x)
%timeit cityblock_numba(x,x)

## Conclusions

Here Numba's `prange` gives us an extra ~50% speed up with only a couple of annotations.